# Classes

Class enables inheritance.

##### 🟢 &nbsp; &nbsp; Constructor & Initializer
In the following code, `class counter` creates an _abbreviation_ called `counter` and it stands for the object type `<get : int; incr : unit>`. Class name is actually the name of the constructor function of the class.

In [88]:
class counter = object
    val mutable _count = 0
    method get = _count
    method incr = _count <- _count + 1
end ;;

let c = new counter ;;
c#get ;;
c#incr ;;
c#get ;;

class counter :
  object val mutable _count : int method get : int method incr : unit end


val c : counter = <obj>


- : int = 0


- : unit = ()


- : int = 1


##### Setting initial value

In [89]:
class counter = fun (init : int) -> object
    val mutable _count = init
    method get = _count   
end ;;

let c = new counter ;;
(c 10)#get ;;

(new counter 20)#get ;;

class point (x: int) (y: int) = object
    val mutable _x = x
    val mutable _y = y
    method get = (_x, _y)
end ;;

let p = new point 1 1 ;;
p#get ;;

(new point 2 2)#get ;;

let f = new point 1 ;;
(f 2)#get ;;

class counter : int -> object val mutable _count : int method get : int end


val c : int -> counter = <fun>


- : int = 10


- : int = 20


class point :
  int ->
  int ->
  object val mutable _x : int val mutable _y : int method get : int * int end


val p : point = <obj>


- : int * int = (1, 1)


- : int * int = (2, 2)


val f : int -> point = <fun>


- : int * int = (1, 2)


##### Initializers : Evaluating expression _before_ or _after_ object is constructed. 

In [218]:
class person ~(name:string) ~(age:float) ?mode (healthy:bool) = 
    let m = match mode with None -> "Normal" | Some x -> x in
    object(self)
        initializer (* can add multiple times *)
            print_string (Printf.sprintf "Name ---> %s" self#get_name); 
            print_newline() ;
        initializer
            print_string (Printf.sprintf "Mode ---> %s" self#get_mode); 
            print_newline() ;
        val _name = name
        val _age = age
        val _mode = m
        val _healthy = healthy
        method get_name = _name
        method get_age = _age
        method get_mode = _mode
        method is_healthy = _healthy        
    end ;;

let p = new person ~name:"Ryan" ~age:6.1 ~mode:"happy!" true;;
p#get_name ;;
p#get_age ;;
p#get_mode ;;
p#is_healthy ;;

class person :
  name:string ->
  age:float ->
  ?mode:string ->
  bool ->
  object
    val _age : float
    val _healthy : bool
    val _mode : string
    val _name : string
    method get_age : float
    method get_mode : string
    method get_name : string
    method is_healthy : bool
  end


Name ---> Ryan
Mode ---> happy!


val p : person = <obj>


- : string = "Ryan"


- : float = 6.1


...

- : bool = true


class virtual base : object val mutable virtual x : int end


class derived : object val mutable x : int end


base initializer...
derived initializer...


val d : derived = <obj>


##### Initializer in inheritance
initializer from parent class will be called first.

In [230]:
class virtual base = object 
    val virtual mutable x : int
    initializer
        print_string "base initializer...";
        print_newline() ;
end ;;

class derived = object
    inherit base
    val mutable x = 11
    initializer
        print_string "derived initializer...";
        print_newline() ;
end ;;

let d = new derived ;;

class virtual base : object val mutable virtual x : int end


class derived : object val mutable x : int end


base initializer...
derived initializer...


val d : derived = <obj>


<br/>

##### 🟢 &nbsp; &nbsp; Class interfaces
Useful for documentation and constraining the type of a class.
Both concrete instance variables and concrete private methods can be hidden by a class type constraint. Public methods and virtual members, however, cannot.

In [133]:
class type ['a] counter_type = object
    method get : 'a
    method incr : unit
    method decr : unit
end ;;

class ['a] counter (init : 'a) = object (self)
    val mutable count = init
    method private set v = count <- v
    method get = count    
    method incr = self#set (count + 1)
    method decr = self#set (count - 1)
end ;;

let c : int counter_type = new counter 1 ;;
c#incr ;;
c#get ;;

class type ['a] counter_type =
  object method decr : unit method get : 'a method incr : unit end


class ['a] counter :
  'a ->
  object
    constraint 'a = int
    val mutable count : 'a
    method decr : unit
    method get : 'a
    method incr : unit
    method private set : 'a -> unit
  end


val c : int counter_type = <obj>


- : unit = ()


- : int = 2


The interface of a class can also be specified in a module signature, and used to restrict the inferred signature of a module.

In [168]:
module type REGISTRY = sig
    class ['a] cell : 'a -> object
        method get : 'a
        method set : 'a -> unit
    end 
end ;;

module Registry : REGISTRY = struct
    class ['a] cell' (init : 'a) = object
        val mutable cell_value = init    
        method get = cell_value
        method set v = cell_value <- v
    end ;;
    
    class ['a] cell  = ['a] cell'
end

let r = new Registry.cell 10 ;;
r#get ;;
r#set 111 ;;
r#get ;;

module type REGISTRY =
  sig
    class ['a] cell :
      'a -> object method get : 'a method set : 'a -> unit end
  end


module Registry : REGISTRY


val r : int Registry.cell = <obj>


- : int = 10


- : unit = ()


- : int = 111


##### 🟢 &nbsp; &nbsp; Parameterized classes

In [102]:
class ['a] counter (init : 'a) = object
    val mutable count = init
    method get = count
end ;;

let c = new counter 10 ;;
c#get ;;

class ['a, 'b] hetro_pair (a : 'a) (b : 'b) = object
    val _a = a
    val _b = b
    method get = (a,b)
end ;;

let p = new hetro_pair 10 9.8 ;;
p#get ;;

class ['a] counter : 'a -> object val mutable count : 'a method get : 'a end


val c : int counter = <obj>


- : int = 10


class ['a, 'b] hetro_pair :
  'a -> 'b -> object val _a : 'a val _b : 'b method get : 'a * 'b end


val p : (int, float) hetro_pair = <obj>


- : int * float = (10, 9.8)


##### constraint clause

In [202]:
class material = object
    method glow c = Printf.sprintf "highlighting with color %s" c
end ;;

class ['a] mesh (m : 'a) = object
    val mutable material = m   
    method highlight = (material#glow : string -> string)  
end ;;

let m = new mesh (object method glow c = Printf.sprintf "highlighting with color %s" c  end) ;;
m#highlight "orange" ;;

let m = new mesh (new material);;
m#highlight "pink" ;;

(* explicitly declare constraint clause *)
class ['a] mesh (m : 'a) = object
   (* #material is an abbreviation for the type of any object implementing the signature of material *)
    constraint 'a = #material
    val mutable material = m   
    method highlight = material#glow
end ;;

let m = new mesh (new material) ;;
m#highlight "blue" ;;

let m = new mesh (object method glow c = Printf.sprintf "highlighting with color %s" c method foo = 42  end) ;;
m#highlight "silver" ;;

(* 
    the following code won't compile
    look at https://discuss.ocaml.org/t/constraint-clause-in-class/10074
*)
class ['a] mesh (m : 'a) = object
   (* material is the exact type of the material class *)
    constraint 'a = material
    val mutable material = m   
    method highlight = material#glow
end ;;

let m = new mesh (object method glow c = Printf.sprintf "highlighting with color %s" c method foo = 42  end) ;;
m#highlight "silver" ;;

class material : object method glow : string -> string end


class ['a] mesh :
  'a ->
  object
    constraint 'a = < glow : string -> string; .. >
    val mutable material : 'a
    method highlight : string -> string
  end


val m : < glow : string -> string > mesh = <obj>


- : string = "highlighting with color orange"


val m : material mesh = <obj>


...

val m : < foo : int; glow : string -> string > mesh = <obj>


- : string = "highlighting with color silver"


class ['a] mesh :
  'a ->
  object
    constraint 'a = #material
    val mutable material : 'a
    method highlight : string -> string
  end


val m : < foo : int; glow : string -> string > mesh = <obj>


- : string = "highlighting with color silver"


##### #T represents any object implementing the signature of class T.

In [273]:
class virtual base = object
    method virtual foo : string
end ;;

class derived = object
    inherit base
    method foo = "derived foo"
end ;;

let call_foo (c : #base) = c#foo ;;
call_foo (new derived) ;;
call_foo (object method foo = "anonymous foo" method bar = "bar" end) ;;

class virtual base : object method virtual foo : string end


class derived : object method foo : string end


val call_foo : #base -> string = <fun>


- : string = "derived foo"


- : string = "anonymous foo"


<br/>

##### 🟢 &nbsp; &nbsp; Virtual Method
- A class containing virtual methods must be flagged virtual, and cannot be instantiated. Similiar to c# abstract class.
- Instance variable can be declared as virtual.

In [209]:
class virtual shape = object
    val mutable virtual _x : float
    val mutable virtual _y : float
    method get_x = _x
    method set_x v = _x <- v
    method get_y = _y
    method set_y v = _y <- v
    method virtual draw :string
end ;;

class circle ~(x : float)  ~(y : float) = object
    inherit shape
    val mutable _x = x
    val mutable _y = y
    method draw = Printf.sprintf "drawing circle at x = %F, y = %F" _x _y
end ;;

let c = new circle ~x:10. ~y:11. ;;
c#draw ;;

class virtual shape :
  object
    val mutable virtual _x : float
    val mutable virtual _y : float
    method virtual draw : string
    method get_x : float
    method get_y : float
    method set_x : float -> unit
    method set_y : float -> unit
  end


class circle :
  x:float ->
  y:float ->
  object
    val mutable _x : float
    val mutable _y : float
    method draw : string
    method get_x : float
    method get_y : float
    method set_x : float -> unit
    method set_y : float -> unit
  end


val c : circle = <obj>


- : string = "drawing circle at x = 10., y = 11."


##### 🟢 &nbsp; &nbsp; Private method
- Private methods are inherited (they are by default visible in subclasses), unless they are hidden by signature matching.
- Private methods can be made public or override in a subclass.

In [217]:
class base = object
    method private display = "display method in base"
end ;;

(* making private method public in derived class *)
class derived_one = object
    inherit base
    method virtual display : _
end ;;
let d = new derived_one ;;
d#display ;;

(* alternatively *)
class derived_one = object (self : < display : _; ..>)
    inherit base
end ;;
let d = new derived_one ;;
d#display ;;

(* overiding private method in derived class *)
class derived_two = object
    inherit base as super
    method display = 
        super#display ^ " ..calling from derived";
end ;;
let dt = new derived_two ;;
dt#display ;;

class base : object method private display : string end


class derived_one : object method display : string end


val d : derived_one = <obj>


- : string = "display method in base"


class derived_one : object method display : string end


val d : derived_one = <obj>


- : string = "display method in base"


class derived_two : object method display : string end


val dt : derived_two = <obj>


- : string = "display method in base ..calling from derived"


##### 🟢 &nbsp; &nbsp; Inheritance
Inheritance is like textual inclusion. If more than one definition for a name, the last definition wins.

In [200]:
class ['a] point2d (x : 'a) (y : 'a) = object(self)
    val mutable _x = x
    val mutable _y = y
    method getX = _x
    method getY = _y
    method setX v = _x <- v
    method setY v = _y <- v
    method set2 (x, y) = self#setX x ; self#setY y
end ;;

class ['a] point3d (x : 'a) (y : 'a) (z : 'a) = object(self)
    inherit ['a] point2d x y as super
    val mutable z = z
    method getZ = z
    method setZ v = z <- v
    method set3 (x, y, z) =
        super#set2 (x, y) ;
        self#setZ z 
end ;;

let p3 = new point3d 1. 2. 3. ;;
p3#getX ;;
p3#getY ;;
p3#getZ ;;

class ['a] point2d :
  'a ->
  'a ->
  object
    val mutable _x : 'a
    val mutable _y : 'a
    method getX : 'a
    method getY : 'a
    method set2 : 'a * 'a -> unit
    method setX : 'a -> unit
    method setY : 'a -> unit
  end


class ['a] point3d :
  'a ->
  'a ->
  'a ->
  object
    val mutable _x : 'a
    val mutable _y : 'a
    val mutable z : 'a
    method getX : 'a
    method getY : 'a
    method getZ : 'a
    method set2 : 'a * 'a -> unit
    method set3 : 'a * 'a * 'a -> unit
    method setX : 'a -> unit
    method setY : 'a -> unit
    method setZ : 'a -> unit
  end


val p3 : float point3d = <obj>


- : float = 1.


- : float = 2.


- : float = 3.


##### explicitly marking as overriding another definition

In [253]:
class base = object
    val x = 1
    method private private_show = "private show"
    method show = "show method in base. x = " ^ string_of_int x
end ;;

class derived = object
    inherit base
    (* if base don't have show method or x instance variable, compiler will throw error *)
    val! x = 1111 
    method! show = "show method in derived. x = " ^ string_of_int x
end ;;

let d = new derived ;;
d#show ;;

class derived_two = object 
    val x = 101   
    method show = "show method in derived_two"
    
    (* override above method and instance variable, 
    base method and instance variable will be used instead  *)
    inherit! base 
end ;;

let dt = new derived_two ;;
dt#show ;;

class base :
  object
    val x : int
    method private private_show : string
    method show : string
  end


class derived :
  object
    val x : int
    method private private_show : string
    method show : string
  end


val d : derived = <obj>


- : string = "show method in derived. x = 1111"


class derived_two :
  object
    val x : int
    method private private_show : string
    method show : string
  end


val dt : derived_two = <obj>


- : string = "show method in base. x = 1"


##### 🟢 &nbsp; &nbsp; Multiple inheritance

In [265]:
class reader = object   
    val v = ref 0
    method get_val = !v
end ;;

class writer = object  
    val v = ref 0   
    method set_val x = v := x  
end ;;

class derived = object
    inherit reader
    inherit! writer
end ;;

let d = new derived ;;
d#set_val 111111 ;;
d#get_val ;;

class reader : object val v : int ref method get_val : int end


class writer : object val v : int ref method set_val : int -> unit end


class derived :
  object
    val v : int ref
    method get_val : int
    method set_val : int -> unit
  end


val d : derived = <obj>


- : unit = ()


- : int = 111111
